In [1]:
# PLOTTER FUNCTION
import imageio
import IPython
import fresnel
import math 

def render_mixed_shapes(snapshot, vispos, box=True):
    # Initialize Fresnel device and tracer
    device = fresnel.Device()
    tracer = fresnel.tracer.Path(device=device, w=500, h=500)
    # Get simulation box size
    L = snapshot.configuration.box[0]
    # Fresnel scene
    scene = fresnel.Scene(device)
    # Define colors for spheres and cubes
    colors = {
        's1': fresnel.color.linear([0.1, 0.7, 0.3]),
        's2': fresnel.color.linear([0.5, 0.9, 0.6]),
        's3': fresnel.color.linear([0.5, 0.7, 0.1]),
        'c': fresnel.color.linear([0.8, 0.1, 0.1])      
    }
    # Define cube vertices (standard cube)
    cube_vertices = [
        [-0.5, -0.5, -0.5], [+0.5, -0.5, -0.5], [+0.5, +0.5, -0.5], [-0.5, +0.5, -0.5],
        [-0.5, -0.5, +0.5], [+0.5, -0.5, +0.5], [+0.5, +0.5, +0.5], [-0.5, +0.5, +0.5],
    ]

    # Process particles
    positions = snapshot.particles.position[:]
    types = snapshot.particles.typeid[:]
    type_names = snapshot.particles.types

    for i, pos in enumerate(positions):
        if type_names[types[i]][0] == 's':
            # Add a sphere
            fresnel.geometry.Sphere(scene, N=1, position=[pos], radius=0.5*1.2,
                        material=fresnel.material.Material(
                        color=colors[type_names[types[i]]], roughness=0.5),
                        outline_width = 0.1)
        elif type_names[types[i]] == 'c':
            # Add a cube
            poly_info = fresnel.util.convex_polyhedron_from_vertices(cube_vertices)
            geometry = fresnel.geometry.ConvexPolyhedron(scene, poly_info, N=1)
            geometry.position[:] = pos
            geometry.material = fresnel.material.Material(
                color=colors['c'], roughness=0.5
            )
            geometry.outline_width = 0.05

    # Add box outline
    if box: fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=0.02)

    # Set up lighting and camera
    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
        fresnel.light.Light(
            direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3
        ),
    ]
    scene.camera = fresnel.camera.Orthographic(
        position=vispos*L, look_at=(0, 0, 0), up=(0, 0, -1), height=L * 1.4 + 1
    )
    scene.background_alpha = 1
    scene.background_color = (1, 1, 1)
    return IPython.display.Image(tracer.sample(scene, samples=500)._repr_png_())

# Function to generate GIF from snapshots
def create_gif(snapshots, view_vector, output_suffix):
    frames = []
    for snap in snapshots:
        # Render each snapshot into a PNG image
        image = render_mixed_shapes(snap, view_vector)
        
        # Convert IPython Image to bytes
        png_data = image.data  # Access the raw PNG data
        
        # Append the PNG frame to the frames list
        frames.append(imageio.imread(png_data))
    
    output_file = f"{filename}-{output_suffix}.gif"
    imageio.mimwrite(output_file, frames, fps=fps)

In [2]:
import hoomd
print(hoomd.version)
print(hoomd.__file__)

Exporting ExternalFieldTypeHarmonic
<module 'hoomd.version' from '/home/dyang4/Software/hoomd-custom/hoomd/version.py'>
/home/dyang4/Software/hoomd-custom/hoomd/__init__.py


In [3]:
# GEOMETRY FUNCTION
import hoomd
import hoomd.hpmc
import numpy as np
from hoomd.write.gsd import GSD
import matplotlib.pyplot as plt

# Define PC lattice positions
def pc_lattice(a, nx, ny, nz):
    """Generate PC lattice points in x, y, z order for cubes facing each other."""
    positions = []
    for z in range(nz):  # Correct order: iterate z first
        for y in range(ny):  # Then iterate y
            for x in range(nx):  # Finally iterate x
                positions.append((x * a, y * a, z * a))
    # Convert to NumPy array and center the box
    positions = np.array(positions)
    center_shift = np.array([(nx - 1) * a / 2, (ny - 1) * a / 2, (nz - 1) * a / 2])
    positions -= center_shift
    return positions

def get_neighbours(index, direction, nx, ny, nz):
    # Convert index to coordinates
    z = index // (nx * ny)
    y = (index % (nx * ny)) // nx
    x = index % nx
           
    # Determine neighbor offset based on direction
    if direction == '+x':
        nx_new = x + 1
        if nx_new < nx:
            return nx_new + nx * y + nx * ny * z
    elif direction == '-x':
        nx_new = x - 1
        if nx_new >= 0:
            return nx_new + nx * y + nx * ny * z
    elif direction == '+y':
        ny_new = y + 1
        if ny_new < ny:
            return x + nx * ny_new + nx * ny * z
    elif direction == '-y':
        ny_new = y - 1
        if ny_new >= 0:
            return x + nx * ny_new + nx * ny * z
    elif direction == '+z':
        nz_new = z + 1
        if nz_new < nz:
            return x + nx * y + nx * ny * nz_new
    elif direction == '-z':
        nz_new = z - 1
        if nz_new >= 0:
            return x + nx * y + nx * ny * nz_new
    else:
        raise ValueError("Invalid direction. Choose from '+x', '-x', '+y', '-y', '+z', '-z'.")
    return None  # If the neighbor does not exist

In [ ]:
# CONSTANTS
K = 0.05   # Temperature
N = 1      # NStart
I = 10000  # Relax Interval
D = 0.005  # Movement
S = 1.0    # Size Ratio
E = 2.5    # Interaction Ratio
B = 11     # BoundarySize
R = 2.5    # Interaction Range
fRatio = 0.8 # CA Selection
L = [(5, 5, 0)]
F = 100    # Force
filename = f"K{int(K*10000)}_F{F}_T{L}_I{I}_S{int(S*100)}"

print(filename)

In [ ]:
def assign_particle_types(snap, L, nx, ny, nz, boundary_size):
    def calculate_linear_index_3d(position, nx, ny, nz):
        """Calculate the linear index in a 3D grid."""
        x, y, z = position
        return x + y * nx + z * nx * ny

    for pos in L:
        x, y, z = pos
        linear_index = calculate_linear_index_3d(pos, nx, ny, nz)
        
        if z == 0 or z == boundary_size - 1:
            snap.particles.typeid[linear_index] = 1  # 's1'
        elif y == 0 or y == boundary_size - 1:
            snap.particles.typeid[linear_index] = 3  # 's3'
        elif x == 0 or x == boundary_size - 1:
            snap.particles.typeid[linear_index] = 2  # 's2'
            
# Initialize device (CPU or GPU)
device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device, seed=0)

# Define the simulation box
ss = S
sc = 1.0
sm = ss / 2 + sc / 2
E0 = 20
nx, ny, nz = B, B, B  # Number of cubes along each axis
N_s = 1
N_c = int(nx * ny * nz) - N_s
N_t = nx * ny * nz
a = 1.0  
positions = pc_lattice(a, nx, ny, nz)

# Create a snapshot with spheres and cubes
snap = hoomd.Snapshot()
snap.particles.N = N_s + N_c
snap.particles.position[:] = positions
snap.particles.types = ['c', 's1', 's2', 's3']
snap.particles.typeid[:N_t] = 0  # Default to 'c' (cube)
assign_particle_types(snap, L, nx, ny, nz, B)

# Define the simulation box
snap.configuration.box = [5 + nx * a, 5 + ny * a, 5 + nz * a, 0, 0, 0]
sim.create_state_from_snapshot(snap)

# Define Integrator
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron(
    default_d=D, default_a=0.01,
    translation_move_probability=1,
    kT=K * np.log(N_t)
)

# Define Shapes
cube_vertices = [
    [-0.5, -0.5, -0.5], [+0.5, -0.5, -0.5], [+0.5, +0.5, -0.5], [-0.5, +0.5, -0.5],
    [-0.5, -0.5, +0.5], [+0.5, -0.5, +0.5], [+0.5, +0.5, +0.5], [-0.5, +0.5, +0.5],
]
mc.shape['c'] = dict(vertices=cube_vertices, sweep_radius=0.0)

# Add identical shapes for s1, s2, and s3
sphere_shape = dict(vertices=[[0, 0, 0]], sweep_radius=0.5)
mc.shape['s1'] = sphere_shape
mc.shape['s2'] = sphere_shape
mc.shape['s3'] = sphere_shape

# Define LJ pair potentials
lj = hoomd.hpmc.pair.LennardJones()
lj.params[( 'c',  'c')] = dict(epsilon=E0 * E, sigma=sc * a / 2**(1 / 6), r_cut=1.2 * a)
lj.params[('s1', 's1')] = dict(epsilon=E0 * 1, sigma=ss * a / 2**(1 / 6), r_cut=R * a)
lj.params[('s2', 's2')] = lj.params[('s1', 's1')]
lj.params[('s3', 's3')] = lj.params[('s1', 's1')]
lj.params[('s1', 's2')] = lj.params[('s1', 's1')]
lj.params[('s1', 's3')] = lj.params[('s1', 's1')]
lj.params[('s2', 's3')] = lj.params[('s1', 's1')]
lj.params[('s1',  'c')] = dict(epsilon=E0 * E, sigma=sm * a / 2**(1 / 6), r_cut=1.2 * a)
lj.params[('s2',  'c')] = lj.params[('s1', 'c')]
lj.params[('s3',  'c')] = lj.params[('s1', 'c')]

mc.pair_potentials = [lj]

# Define Walls along Z direction, Deprecated
if False:
    x_min = positions[:, 0].min() - 0.5  # Minimum z-coordinate
    x_max = positions[:, 0].max() + 0.5  # Maximum z-coordinate
    y_min = positions[:, 1].min() - 0.5  # Minimum z-coordinate
    y_max = positions[:, 1].max() + 0.5  # Maximum z-coordinate
    z_min = positions[:, 2].min() - 0.5  # Minimum z-coordinate
    z_max = positions[:, 2].max() + 0.5  # Maximum z-coordinate

    walls = [
        hoomd.wall.Plane(origin=(0, 0, z_min), normal=( 0,  0,  1), open=True),
        hoomd.wall.Plane(origin=(0, 0, z_max), normal=( 0,  0, -1), open=True),
        hoomd.wall.Plane(origin=(0, y_min, 0), normal=( 0,  1,  0), open=True),
        hoomd.wall.Plane(origin=(0, y_max, 0), normal=( 0, -1,  0), open=True),
        hoomd.wall.Plane(origin=(x_min, 0, 0), normal=(-1,  0,  0), open=True),
        hoomd.wall.Plane(origin=(x_max, 0, 0), normal=(-1,  0,  0), open=True),
    ]
    wall_pots = hoomd.hpmc.external.WallPotential(walls)
    mc.external_potentials = [wall_pots]

if True: # Customized Harmonic, Need Recompile
    print(snap.particles.types)
    k_translational = {
        'c':  (0, 0, 0), # No Force
        's1': (0, 0, F), # Enhanced Penalty Along Z
        's3': (0, F, 0),
        's2': (F, 0, 0)
    }
    
    # Define reference positions
    reference_positions = positions.copy().astype(np.float64)
    print(reference_positions.dtype, reference_positions.shape)
    
    # Create the external potential
    external_potential = hoomd.hpmc.external.TypeHarmonic(k_translational=k_translational, 
                                      reference_positions=reference_positions)
    mc.external_potentials = [external_potential]
    

# Define Logger
logger = hoomd.logging.Logger()
logger.add(lj, quantities=['energy'])

# Assign the integrator to the simulation
sim.operations.integrator = mc

# Run the simulation and test
init_snapshot = sim.state.get_snapshot()
sim.run(1)

# Get initial positions for all particles
with sim.state.cpu_local_snapshot as snap:
    typeid_array = np.array(snap.particles.typeid)
    positions = np.array(snap.particles.position)

# Store the initial positions for reference
initial_positions = positions.copy()

fig = render_mixed_shapes(init_snapshot, np.array([1, -1, -1]))
display(fig)

In [ ]:
from IPython.display import display, clear_output
import random
import time
import sys

directions = ['+x', '-x', '+y', '-y', '+z', '-z', 'F']

direction_probabilities = {
    's1': {  # Probabilities for s1 propagation
        '+x': 0.225, '-x': 0.225, '+y': 0.225, '-y': 0.225, '+z': 0.05*(1-fRatio), '-z': 0.05*(1-fRatio), 'F': 0.1*fRatio,
    },
    's2': {  # Probabilities for s2 propagation
        '+y': 0.225, '-y': 0.225, '+z': 0.225, '-z': 0.225, '+x': 0.05*(1-fRatio), '-x': 0.05*(1-fRatio), 'F': 0.1*fRatio,
    },
    's3': {  # Probabilities for s3 propagation
        '+z': 0.225, '-z': 0.225, '+x': 0.225, '-x': 0.225, '+y': 0.05*(1-fRatio), '-y': 0.05*(1-fRatio), 'F': 0.1*fRatio,
    },
}

# Initialize valid_tag with all particle tags where typeid != 0 (non-cube particles)
with sim.state.cpu_local_snapshot as data:
    typeid_array = data.particles.typeid
    particle_tags = data.particles.tag  # Tags for consistent referencing
    valid_tag = [particle_tags[i] for i, t in enumerate(typeid_array) if t != 0]

# Initialize a set for deleted tags
deleted_tag = set()

print(f"{'Step':<10}{'TPS':<10}{'Accept':<10}{'Energy':<15}", flush=True)
snapshots = [sim.state.get_snapshot()]
stopcount = None

while sim.timestep <= 1000000:
    sim.run(1000)
    # Log simulation statistics
    energy = lj.energy
    counters = mc.counters
    trans_accepted, trans_rejected = counters.translate
    accept_ratio = trans_accepted / (trans_accepted + trans_rejected)
    print(f"{sim.timestep:<10}{sim.tps:<10.2f}{accept_ratio:<10.3f}{energy:<15.3f}", flush=True)

    # Every I timesteps, save a snapshot and visualize
    if sim.timestep % I == 1:
        snap = sim.state.get_snapshot()
        snapshots.append(snap)

        clear_output(wait=True)
        fig = render_mixed_shapes(snapshots[-1], np.array([1, -1, -1]))
        display(fig)

        # Propagation logic
        with sim.state.cpu_local_snapshot as data:
            typeid_array = data.particles.typeid
            particle_tags = data.particles.tag

            # Filter valid_tag to exclude deleted tags
            valid_tag = [particle_tags[i] for i, t in enumerate(typeid_array) if t != 0]
            valid_tag = [tag for tag in valid_tag if tag not in deleted_tag]
            if len(valid_tag) == 0:
                if stopcount is None:
                    stopcount = 0
                else:
                    stopcount += 1
            if stopcount is not None and stopcount >= 5:
                break  
            
            while valid_tag:
                # Select a random particle tag from valid_tag
                current_tag = random.choice(valid_tag)
            
                # Find the index of the current tag
                current_index = np.where(particle_tags == current_tag)[0][0]
                current_type = typeid_array[current_index]
            
                # Skip propagation if the type is invalid (e.g., already deleted)
                if current_type not in [1, 2, 3]:
                    valid_tag.remove(current_tag)
                    continue
            
                valid_directions = []
            
                # Find valid neighbors for propagation based on the current type
                for direction in directions:
                     if direction == 'F': 
                         neighbor_tag = current_tag
                     else:
                         neighbor_tag = get_neighbours(current_tag, direction, nx, ny, nz)
                     if neighbor_tag is not None and neighbor_tag not in deleted_tag:
                        # Find neighbor index inside the context manager
                        neighbor_index = np.where(particle_tags == neighbor_tag)[0][0]
                        neighbor_type = typeid_array[neighbor_index]
            
                        # Propagation rule: Only propagate to 'c' (type 0)
                        if neighbor_type == 0 or direction == 'F':  # If neighbor is a cube
                            valid_directions.append((direction, neighbor_tag))

                if not valid_directions or len(valid_directions)==1 :
                    # If no valid neighbors (ONLY F), remove the current tag
                    valid_tag.remove(current_tag)
                    deleted_tag.add(current_tag)
                else:
                    # Get the appropriate probabilities for the current type
                    direction_probs = direction_probabilities[f's{current_type}']           
                    # Renormalize probabilities for the current valid directions
                    valid_directions_with_prob = [
                            (direction, neighbor_tag, direction_probs[direction])
                            for direction, neighbor_tag in valid_directions
                    ]
                    
                    total_prob = sum(prob for _, _, prob in valid_directions_with_prob)
                    normalized_probs = [prob / total_prob for _, _, prob in valid_directions_with_prob]
            
                    # Select a random valid direction based on weighted probabilities
                    chosen_direction, neighbor_tag = random.choices(
                            population=[(direction, neighbor_tag) for direction, neighbor_tag, _ in valid_directions_with_prob],
                            weights=normalized_probs
                    )[0]
                    
                    # Update the neighbor's typeid to the current particle's type
                    neighbor_index = np.where(particle_tags == neighbor_tag)[0][0]
                    typeid_array[neighbor_index] = current_type
            
                    valid_tag.remove(current_tag)

                    #print(chosen_direction, valid_directions_with_prob, normalized_probs)

In [ ]:
# Parameters
fps = 3  # Frames per second for the movie

# Save the XYZ file
xyz_file = f"{filename}.xyz"
with open(xyz_file, 'w') as xyz:
    for snap_idx, snap in enumerate(snapshots):
        xyz.write(f"{len(snap.particles.typeid)}\nFrame {snap_idx}\n")
        for j, _ in enumerate(snap.particles.typeid):
            xyz.write(f"{snap.particles.typeid[j]} {snap.particles.position[j][0]} {snap.particles.position[j][1]} {snap.particles.position[j][2]}\n")
            
# Generate Front and Back GIFs
# create_gif(snapshots, np.array([1, -1, -1]), "Front")
create_gif(snapshots, np.array([-1, 1, -1]), "Back")